In [ ]:
#default_exp controls

# Controls

> In this module, I implement the physical controls for MusicPlayer. Borrowed a lot from [Freenove ADC Module](https://github.com/Freenove/Freenove_Ultimate_Starter_Kit_for_Raspberry_Pi/blob/master/Code/Python_Code/07.1.1_ADC/ADC.py).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import time
import smbus
from IPython.display import clear_output
from time_travel_radio.core import *
import numpy as np

In [ ]:
#export
class ADCDevice(object):
    def __init__(self):
        self.cmd = 0
        self.address = 0
        self.bus=smbus.SMBus(1)
        # print("ADCDevice init")
        
    def detectI2C(self,addr):
        try:
            self.bus.write_byte(addr,0)
            print("Found device in address 0x%x"%(addr))
            return True
        except:
            print("Not found device in address 0x%x"%(addr))
            return False
            
    def close(self):
        self.bus.close()

class ADS7830(ADCDevice):
    def __init__(self):
        super(ADS7830, self).__init__()
        self.cmd = 0x84
        self.address = 0x4b # 0x4b is the default i2c address for ADS7830 Module.   
        
    def analogRead(self, chn): # ADS7830 has 8 ADC input pins, chn:0,1,2,3,4,5,6,7
        value = self.bus.read_byte_data(self.address, self.cmd|(((chn<<2 | chn>>1)&0x07)<<4))
        return value

In [ ]:
#export
adc = ADCDevice() # Define an ADCDevice class object

def setup():
    global adc
    if(adc.detectI2C(0x4b)): # Detect the ads7830
        adc = ADS7830()
    else:
        print("No correct I2C address found, \n"
        "Please use command 'i2cdetect -y 1' to check the I2C address! \n"
        "Program Exit. \n");
        exit(-1)
        
def loop():
    while True:
        value = adc.analogRead(0)    # read the ADC value of channel 0
        voltage = value / 255.0 * 3.3  # calculate the voltage value
        clear_output(wait=True)
        print ('ADC Value : %d, Voltage : %.2f'%(value,voltage))
        time.sleep(0.1)

def destroy():
    adc.close()
    

In [ ]:
#export
if __name__ == '__main__':   # Program entrance
    print ('Program is starting ... ')
    try:
        setup()
        loop()
    except KeyboardInterrupt: # Press ctrl-c to end the program.
        destroy()

Program is starting ... 
Not found device in address 0x4b
No correct I2C address found, 
Please use command 'i2cdetect -y 1' to check the I2C address! 
Program Exit. 



OSError: [Errno 9] Bad file descriptor

## Controlling MusicPlayer With Potentiometer

In [ ]:
player = DecadesPlayer("Brandon's Raspberry Pi")

Find range of values for each playlist:

In [ ]:
bin_size = 255/len(player.playlists); bin_size 

25.5

In [ ]:
bins = []
val = bin_size
while val <= 255:
    bins.append(val)
    val += bin_size
bins, len(bins)

([25.5, 51.0, 76.5, 102.0, 127.5, 153.0, 178.5, 204.0, 229.5, 255.0], 10)

In [ ]:
np.digitize(20, bins), np.digitize(240, bins)

(0, 9)

**TODO**: Refactor this to make `decades` be a method of `DecadesPlayer` or something.

In [ ]:
decades = [dec for dec in DecadesPlayer.playlists]

In [ ]:
decades[0]

'1920s'

In [ ]:
def read_decade():
    """Reads value from potentiometer and returns the corresponding decade."""
    value = adc.analogRead(0)  # B/c potentiometer is connected to channel 0 of ADC
    bucket = np.digitize(value, bins)  # Returns indice of bin that value is in
    return decades[bucket]

In [ ]:
try:
    setup()
    old_decade = read_decade()
    while True:
        new_decade = read_decade()
        if new_decade != old_decade:
            print(f"Playing the {new_decade}")
            player.play_music(new_decade)
            old_decade = new_decade
        time.sleep(0.01)
except KeyboardInterrupt:
    destroy()

Found device in address 0x4b
Playing the 1930s
Playing the 1920s
Playing the 1930s
Playing the 1940s
Playing the 1930s
Playing the 1920s
Playing the 1930s


Calling `destroy` when the user exits is very important to put the GPIO pins back into a safe state.

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_controls.ipynb.
Converted index.ipynb.
